### Imports

In [3]:
import numpy as np
from pandas import to_datetime, Timedelta
from importlib import import_module

daycount_dict = {'actual/360': 'actual360', 
                 'actual/365': 'actual365', 
                 'actual/actual': 'actualactual', 
                 'thirty/360': 'thirty360'}

### Add-On Rate

The following rates are generally quoted on an add-on rate basis:
- Commercial bank loans and deposits
- Certificates of deposit (CD)
- Repos
- Fed funds
- LIBOR
- SOFR
- Commercial paper (CP) in Euromarkets

Add-on rates are logical and follow simple interest calculations. The **interest is added on to the principal amount** to get the **redemption payment** at maturity.

$$FV=PV+\Big[PV \times AOR \times \frac{Days}{Year}\Big]$$
$$FV=PV\times\Big[1 + \Big(AOR \times \frac{Days}{Year}\Big)\Big]$$

- AOR is the quoted add-on rate
- PV the present value (the initial principal amount)
- FV the future value (the redemption payment including interest)
- Days the number of days until maturity, and 
- Year the number of days in the year

In [25]:
def addon_fv(pv, aor, settle, mat, daycount='actual/360', year=360):
    name = 'isda_daycounters.'+ daycount_dict[daycount]
    days = import_module(name).day_count(settle, mat)
    return pv * (1 + (aor * days  / year))

pv = 1_000_000
aor = 0.039
daycount = 'actual/360'
settle = to_datetime('2025-10-15')
mat = to_datetime('2026-04-13')

fv = addon_fv(pv, aor, settle, mat)
print(f'fv ={round(fv, 2): ,}')

fv = 1,019,500.0


In [28]:
def addon_fv(pv, aor, settle, mat, daycount='actual/360'):
    name = 'isda_daycounters.'+ daycount_dict[daycount]
    days = import_module(name).day_count(settle, mat)
    year = import_module(name).year_fraction(settle, mat) * days
    return pv * (1 + (aor * days  / year))

pv = 1_000_000
aor = 0.039
daycount = 'actual/360'
settle = to_datetime('2025-10-15')
mat = to_datetime('2026-04-13')

fv = addon_fv(pv, aor, settle, mat)
print(f'fv ={round(fv, 2): ,}')

fv = 1,078,000.0


### Discount Rate

The following rates are generally quoted on a discount rate basis:
- T-Bills
- Commercial paper (CP) 
- Banker's acceptance